In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score
import torch
import os
import seaborn as sns
from torch.utils.data import DataLoader,TensorDataset
import pickle
import json
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
from sae.preprocess import get_meta_info,get_freq_emmbeddings, get_tf_idfs_emmbeddings,get_bert_embeddings
from sae.models import DEEPmodel_2
import time
import joblib
%autoreload 2

In [7]:
data_valid = pd.read_csv('books_data_reviews.csv')
data_valid

,Id,Titre,Prix,User_id,Nom lecteur,revue/utilité,revue/score,revue/heure,revue/résumé,revue/texte,Description,Auteurs,Image,Lien Google,Editeur,Date publication,infoLink,Genre,Nb scores
0,B00007FYF3,Visual Basic 6 Master Reference,NaN,A2IVAIN746RPZX,Robert J. McCorkle,7/7,4.0,959385600,Good as far as it goes,The other reviews sum up the book nicely. It i...,Visual Basic 6 Master Reference includes expla...,['Clayton Walnum'],http://books.google.com/books/content?id=pFcHA...,http://books.google.nl/books?id=pFcHAAAACAAJ&d...,Wiley,03/03/1999,http://books.google.nl/books?id=pFcHAAAACAAJ&d...,['Computers'],NaN
1,B00007FYF3,Visual Basic 6 Master Reference,NaN,NaN,NaN,4/4,4.0,948931200,I saw this in the bookstore and...,I saw this book at the local bookstore and hav...,Visual Basic 6 Master Reference includes expla...,['Clayton Walnum'],http://books.google.com/books/content?id=pFcHA...,http://books.google.nl/books?id=pFcHAAAACAAJ&d...,Wiley,03/03/1999,http://books.google.nl/books?id=pFcHAAAACAAJ&d...,['Computers'],NaN
2,B00007FYF3,Visual Basic 6 Master Reference,NaN,A2VK2INRPZQI1S,Steve Mazer,2/2,4.0,953251200,Exactly what I was looking for,This book is exactly what I was looking for. I...,Visual Basic 6 Master Reference includes expla...,['Clayton Walnum'],http://books.google.com/books/content?id=pFcHA...,http://books.google.nl/books?id=pFcHAAAACAAJ&d...,Wiley,03/03/1999,http://books.google.nl/books?id=pFcHAAAACAAJ&d...,['Computers'],NaN
3,B00007FYF3,Visual Basic 6 Master Reference,NaN,NaN,NaN,2/2,4.0,935625600,"Great reference, but that's all.",As a student I find the Complete Reference a u...,Visual Basic 6 Master Reference includes expla...,['Clayton Walnum'],http://books.google.com/books/content?id=pFcHA...,http://books.google.nl/books?id=pFcHAAAACAAJ&d...,Wiley,03/03/1999,http://books.google.nl/books?id=pFcHAAAACAAJ&d...,['Computers'],NaN
4,B00007FYF3,Visual Basic 6 Master Reference,NaN,NaN,NaN,2/2,2.0,922492800,Not very complete. Too many ActiveX controls m...,Don't believe the book's cover. The reference ...,Visual Basic 6 Master Reference includes expla...,['Clayton Walnum'],http://books.google.com/books/content?id=pFcHA...,http://books.google.nl/books?id=pFcHAAAACAAJ&d...,Wiley,03/03/1999,http://books.google.nl/books?id=pFcHAAAACAAJ&d...,['Computers'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,0060982020,You've Got to Read This: Contemporary American...,NaN,A13A9TDQ1UCV3S,"Kenny A. Chaffin ""Seeking Beginner's Mind""",0/0,5.0,1319241600,Wonderful!,Incredible selection of stories. Highly recomm...,"""An exciting new anthology of short fiction ch...","['Ron Hansen', 'Jim Shepard']",http://books.google.com/books/content?id=7XAbA...,http://books.google.nl/books?id=7XAbAQAAIAAJ&q...,Harper Perennial,1994,http://books.google.nl/books?id=7XAbAQAAIAAJ&d...,['Short stories'],NaN
1206,0060982020,You've Got to Read This: Contemporary American...,NaN,A1H78PWTJNTZ83,"Paul ""Pablo""",0/0,4.0,1154044800,"Excellent Variety, Neat Format",This was a textbook for an online course I too...,"""An exciting new anthology of short fiction ch...","['Ron Hansen', 'Jim Shepard']",http://books.google.com/books/content?id=7XAbA...,http://books.google.nl/books?id=7XAbAQAAIAAJ&q...,Harper Perennial,1994,http://books.google.nl/books?id=7XAbAQAAIAAJ&d...,['Short stories'],NaN
1207,0060982020,You've Got to Read This: Contemporary American...,NaN,NaN,NaN,0/0,4.0,876441600,Sometimes it makes me wonder...,The concept of this book really interests me. ...,"""An exciting new anthology of short fiction ch...","['Ron Hansen', 'Jim Shepard']",http://books.google.com/books/content?id=7XAbA...,http://books.google.nl/books?id=7XAbAQAAIAAJ&q...,Harper Perennial,1994,http://books.google.nl/books?id=7XAbAQAAIAAJ&d...,['Short stories'],NaN
1208,0060982020,You've Got to Read This: Contemporary American...,NaN,NaN,NaN,5/8,5.0,916012800,Great Way to Rediscover Authors,I really enjoyed this book. Even if I disliked...,"""An excit

In [8]:
data_valid = pd.read_csv('books_data_reviews.csv')
print(len(data_valid))
data_valid.rename(columns={'revue/score':'rating',"revue/résumé":"summary","revue/texte":"comment"}, inplace=True)
data_valid["summary"] = np.where(pd.isnull(data_valid["summary"]), '', data_valid["summary"])
data_valid["comment"] = np.where(pd.isnull(data_valid["comment"]), '', data_valid["comment"])
len(data_valid)

1210


1210

# Cacul des metas attribut

In [9]:
#on recharge tous les fichiers json pour verifier
with open('titre_info.json', 'r') as fp:
    titre_info = json.load(fp)
with open('author_data.json', 'r') as fp:
    author_data = json.load(fp)
with open('genre_data.json', 'r') as fp:
    genre_data = json.load(fp)
with open('editor_data.json', 'r') as fp:
    editor_data = json.load(fp)


In [10]:

ratings_author, nb_rat_auhtor, ratings_genre, nb_rat_genre, ratings_editor, nb_rat_editor =  get_meta_info(data_valid,titre_info, author_data,genre_data,editor_data)

In [11]:
#on rajoute les colonnes dans le dataframe ratings
data_valid['auteur_note'] = ratings_author
data_valid['auteur_nb'] = nb_rat_auhtor
data_valid['genre_note'] = ratings_genre
data_valid['genre_nb'] = nb_rat_genre
data_valid['editor_note'] = ratings_editor
data_valid['editor_nb'] = nb_rat_editor


In [12]:
X_meta = data_valid[['auteur_note','auteur_nb','genre_note','genre_nb','editor_note','editor_nb']].values

X_meta.shape


(1210, 6)

# Calcul des embeddings

In [13]:

Y_valid_rating = data_valid['rating'].values.astype(int)
Y_valid_sentiment = data_valid['rating'].values.astype(int)
Y_valid_sentiment[Y_valid_sentiment < 3] = 0
Y_valid_sentiment[Y_valid_sentiment == 3] = 1
Y_valid_sentiment[Y_valid_sentiment > 3] = 2
mode = "summary"

# Embeding

## mots simple

In [14]:
Freqmodel = joblib.load('freq_model.pkl')

In [15]:
X_Freq,inutile = get_freq_emmbeddings(list(data_valid[mode].values),model=Freqmodel)
X_valid_Freq = np.concatenate((X_meta,X_Freq),axis=1)
X_Freq

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### SKlearn

In [16]:
# model sentiment
nb_model = joblib.load('freq_gradientboosting.pkl')
result=nb_model.predict(X_valid_Freq)
print(result)
print(Y_valid_sentiment)
accuracy_score(result,Y_valid_sentiment)

[2 2 2 ... 2 2 2]
[2 2 2 ... 2 2 2]


0.756198347107438

In [17]:
# model rating
nb_model = joblib.load('Re_freq_gradientboosting.pkl')
result=nb_model.predict(X_valid_Freq)
print(result)
print(Y_valid_rating)
accuracy_score(result,Y_valid_rating)

[4. 5. 5. ... 5. 5. 5.]
[4 4 4 ... 4 5 4]


0.5644628099173554

### deep

#### sentiment

In [18]:
X_valid_Freq = np.concatenate((X_meta,X_Freq),axis=1)
batch_size = 128
valid_dataset_Freq_sent = TensorDataset(torch.tensor(X_valid_Freq),torch.tensor(Y_valid_sentiment, dtype=torch.long))
valid_loader_Freq_sent = DataLoader(valid_dataset_Freq_sent, batch_size=batch_size, shuffle=False)

In [19]:
modeldSimple = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=3)

modeldSimple.load_state_dict(torch.load('Deep_freq.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [20]:
modeldSimple.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_Freq_sent):
        x = x.float()  
        y_pred_batch = modeldSimple(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_sentiment, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.7966942148760331


#### reting

In [21]:
X_valid_Freq = np.concatenate((X_meta,X_Freq),axis=1)
batch_size = 128
valid_dataset_Freq_sent = TensorDataset(torch.tensor(X_valid_Freq),torch.tensor(Y_valid_rating, dtype=torch.long))
valid_loader_Freq_sent = DataLoader(valid_dataset_Freq_sent, batch_size=batch_size, shuffle=False)

In [22]:
modeldSimple = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=5)

modeldSimple.load_state_dict(torch.load('RE_Deep_freq.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [23]:
modeldSimple.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_Freq_sent):
        x = x.float()  
        y_pred_batch = modeldSimple(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_rating-1, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.5991735537190083


## tf idf

In [24]:
idfmodel = joblib.load('tfidf_model.pkl')

In [25]:
X_idf,inutile = get_tf_idfs_emmbeddings(list(data_valid[mode].values),model=idfmodel)
X_valid_idf = np.concatenate((X_meta,X_idf),axis=1)
X_idf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Sklearn

In [26]:
# Charger le modèle
nb_model = joblib.load('idf_gradientboosting.pkl')
result=nb_model.predict(X_valid_idf)
print(result)
print(Y_valid_sentiment)
accuracy_score(result,Y_valid_sentiment)

[2 2 2 ... 2 2 2]
[2 2 2 ... 2 2 2]


0.759504132231405

In [27]:
# model rating
nb_model = joblib.load('Re_idf_gradientboosting.pkl')
result=nb_model.predict(X_valid_Freq)
print(result)
print(Y_valid_rating)
accuracy_score(result,Y_valid_rating)

[4. 5. 5. ... 5. 5. 5.]
[4 4 4 ... 4 5 4]


0.5636363636363636

### deep

#### sentiment

In [28]:
batch_size = 128
valid_dataset_idf_sent = TensorDataset(torch.tensor(X_valid_idf),torch.tensor(Y_valid_sentiment, dtype=torch.long))
valid_loader_idf_sent = DataLoader(valid_dataset_idf_sent, batch_size=batch_size, shuffle=False)

In [29]:
modeldidf = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=3)
modeldidf.load_state_dict(torch.load('Deep_idf.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [30]:
modeldidf.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_idf_sent):
        x = x.float()  
        y_pred_batch = modeldidf(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_sentiment, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.7958677685950413


#### rating

In [31]:
batch_size = 128
valid_dataset_idf_sent = TensorDataset(torch.tensor(X_valid_idf),torch.tensor(Y_valid_rating, dtype=torch.long))
valid_loader_idf_sent = DataLoader(valid_dataset_idf_sent, batch_size=batch_size, shuffle=False)

In [32]:
modeldidf = DEEPmodel_2(input_size=1006, features = [512,512,512,512,256], sub_features =[256,256], nb_class=5)
modeldidf.load_state_dict(torch.load('Re_Deep_idf.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [33]:
modeldSimple.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_Freq_sent):
        x = x.float()  
        y_pred_batch = modeldSimple(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_rating-1, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.5991735537190083


## bert

In [34]:
X_bert = get_bert_embeddings(list(data_valid[mode].values)).numpy()
X_valid_bert = np.concatenate((X_meta,X_bert),axis=1)

**********


### SKlearn

In [35]:
# Charger le modèle
nb_model = joblib.load('bert_gradientboosting.pkl')
result=nb_model.predict(X_valid_bert)
print(result)
print(Y_valid_sentiment)
accuracy_score(result,Y_valid_sentiment)


[1 2 2 ... 2 2 2]
[2 2 2 ... 2 2 2]


0.7966942148760331

### deep

In [36]:

batch_size = 128
valid_dataset_bert_sent = TensorDataset(torch.tensor(X_valid_bert),torch.tensor(Y_valid_sentiment, dtype=torch.long))
valid_loader_bert_sent = DataLoader(valid_dataset_bert_sent, batch_size=batch_size, shuffle=False)

In [37]:
model = DEEPmodel_2(input_size=774, features = [512,512,512,512,256], sub_features =[256,256], nb_class=3)
model.load_state_dict(torch.load('Deep_bert.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [38]:
model.eval()
with torch.no_grad():
    y_pred = []
    for i, (x, y) in enumerate(valid_loader_bert_sent):
        x = x.float()  
        y_pred_batch = model(x).argmax(1)
        y_pred.append(y_pred_batch)  # Move to CPU for evaluation
    
    y_pred = torch.cat(y_pred)
acc_valid = accuracy_score(Y_valid_sentiment, y_pred.numpy())
print("accuracyh valid :", acc_valid)

accuracyh valid : 0.8487603305785124
